#import Library

In [1]:
import numpy as np
from pyspark.mllib.linalg import Vectors

#Dense vetcor

In [4]:
#使用list
x = [1,2,3,4,5]
dense_x = Vectors.dense(x)
dense_x

DenseVector([1.0, 2.0, 3.0, 4.0, 5.0])

# Spars vector

In [8]:
#三種產生Sparse vector的方法

sparse_x = Vectors.sparse(4, {1: 1.0, 3: 5.5})
print("sparse_x = " + str(sparse_x))

sparse_y = Vectors.sparse(4, [(1, 1.0), (3, 5.5)])
print("sparse_y = " + str(sparse_y))

sparse_z = Vectors.sparse(4, [1, 3], [1.0, 5.5])
print("sparse_z = " + str(sparse_z))

sparse_x = (4,[1,3],[1.0,5.5])
sparse_y = (4,[1,3],[1.0,5.5])
sparse_z = (4,[1,3],[1.0,5.5])
